# Taller 8 - Importando el Mundo: Visualización y Conversión de Formatos 3D

# Importar Librerias necesarias:

In [1]:
!pip install trimesh open3d pyassimp

In [2]:
import trimesh
import numpy as np
import open3d as o3d
import os

#Para visualizar los datos de los modelos:
import pandas as pd


In [3]:
#Liberias para la visualización:

!pip install plotly
import plotly.graph_objects as go


# Subir los archivos en formato gltf, obj y stl.

In [4]:
from google.colab import files

uploaded = files.upload()


Saving PhoneGLTF.gltf to PhoneGLTF.gltf
Saving PhoneOBJ.obj to PhoneOBJ.obj
Saving PhoneSTL.stl to PhoneSTL.stl
Saving scene.bin to scene.bin


In [5]:
# Función para cargar un modelo 3D con trimesh desde una ruta de archivo
def load_model_trimesh(file_path):
    # Se fuerza a cargar el archivo como un objeto de malla (Mesh)
    mesh = trimesh.load(file_path, force='mesh')
    return mesh

# Función para analizar las propiedades estructurales de la malla 3D
def analyze_mesh(mesh):
    print("▶️ Información del modelo:")

    # Imprimir número de vértices y caras (triángulos/polígonos)
    print(f"- Vértices: {len(mesh.vertices)}")
    print(f"- Caras: {len(mesh.faces)}")

    # Verificar si existen normales por vértice
    has_normals = (
        hasattr(mesh, 'vertex_normals') and
        mesh.vertex_normals is not None and
        len(mesh.vertex_normals) == len(mesh.vertices)
    )
    print(f"- Normales: {'Sí' if has_normals else 'No'}")

    # Verificar si hay vértices duplicados (comparando con una versión redondeada)
    unique_vertices = np.unique(mesh.vertices.round(decimals=5), axis=0)
    duplicated = len(mesh.vertices) != len(unique_vertices)
    print(f"- Vértices duplicados: {'Sí' if duplicated else 'No'}")

    # Mostrar el tipo de objeto cargado (por ejemplo, Trimesh o Scene)
    print(f"- Formato detectado: {type(mesh)}")

    # Devolver resumen estructurado como diccionario
    return {
        "vertices": len(mesh.vertices),
        "faces": len(mesh.faces),
        "has_normals": has_normals,
        "duplicates": duplicated
    }

In [6]:
# Función para visualizar una malla 3D usando Plotly

def plot_trimesh_plotly(mesh):
    # Extraer vértices (coordenadas 3D) y caras (índices de triángulos) del objeto de malla
    faces = mesh.faces
    vertices = mesh.vertices

    # Separar las coordenadas X, Y, Z de los vértices
    x, y, z = vertices.T  # Transponer para obtener listas separadas por coordenada

    # Separar los índices i, j, k que definen cada triángulo (caras)
    i, j, k = faces.T

    # Crear una figura 3D con Mesh3d de Plotly
    fig = go.Figure(data=[
        go.Mesh3d(
            x=x, y=y, z=z,      # Coordenadas de los vértices
            i=i, j=j, k=k,      # Índices que forman los triángulos
            opacity=0.5,        # Transparencia para mejor visualización
            color='lightblue'   # Color del modelo
        )
    ])

    # Configurar la vista de la escena
    fig.update_layout(
        title="Visualización del modelo 3D",
        scene=dict(aspectmode='data')  # Mantiene proporciones realistas
    )

    # Mostrar la figura
    fig.show()

# Visualizar objetos:

In [11]:
# Nombres de archivo (ajústalos a los que subiste)
model_files = {
    "OBJ": "PhoneOBJ.obj",
    "STL": "PhoneSTL.stl",
    "GLTF": "PhoneGLTF.gltf"
}

results = {}


for name, path in model_files.items():
    print("____________________________________")
    print(f"\n Modelo: {name}")
    mesh = load_model_trimesh(path)
    results[name] = analyze_mesh(mesh)
    plot_trimesh_plotly(mesh)  #Visualización inline con Plotly


____________________________________

 Modelo: OBJ
▶️ Información del modelo:
- Vértices: 1132
- Caras: 1920
- Normales: Sí
- Vértices duplicados: No
- Formato detectado: <class 'trimesh.base.Trimesh'>


____________________________________

 Modelo: STL
▶️ Información del modelo:
- Vértices: 1132
- Caras: 1920
- Normales: Sí
- Vértices duplicados: No
- Formato detectado: <class 'trimesh.base.Trimesh'>


____________________________________

 Modelo: GLTF
▶️ Información del modelo:
- Vértices: 1778
- Caras: 1920
- Normales: Sí
- Vértices duplicados: Sí
- Formato detectado: <class 'trimesh.base.Trimesh'>


In [12]:
# Convertir OBJ a STL, GLTF, etc.
source_path = "PhoneOBJ.obj"
mesh = load_model_trimesh(source_path)

# Guardar en otros formatos
mesh.export("converted_from_obj.stl")
mesh.export("converted_from_obj.glb")  # GLB binario
mesh.export("converted_from_obj.ply")  # Otro formato útil

print("Conversión completada. Archivos exportados!!!!!")


Conversión completada. Archivos exportados!!!!!


# Bonus: comparación entre varios modelos.

In [13]:
df = pd.DataFrame(results).T
df["has_normals"] = df["has_normals"].map({True: "Yes", False: "No"})
df["duplicates"] = df["duplicates"].map({True: "Yes", False: "No"})

df.style.set_caption("Comparación de Modelos 3D").background_gradient(cmap='YlGn')


,vertices,faces,has_normals,duplicates
OBJ,1132,1920,Yes,No
STL,1132,1920,Yes,No
GLTF,1778,1920,Yes,Yes
